In [13]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

In [14]:
from pyomo.environ import *
import numpy as np

In [15]:
obj_coefs = np.array([-9.6, -2.85, 41.5, 33.05, -0.75, 20.5, -0.35, -31.02, 50.65, 6.125])

const_coefs = np.array([[0, 1, -2, 0, 1, -2, 0, 1, 1, -1],[-1 ,1 ,-3 ,2.5, 0 ,1.4, 0, -2, 1, -1.5],[-0.45 ,4.78, -2, -3.22, -1.3, 1, -1, 0, 0, 0.25],[1, 0, 0, 0, 1, 1, 0, 2, 3, 0]])
const_rhs = np.array([35.4, 47.9, 41.43, 22.7])

In [16]:
lb = np.array([-np.inf, -np.inf, 0, 0, -67, 0, -23, -5, 6, 1])
rb = np.array( [4, np.inf, np.inf, 2, 20, 55, 23, 15, np.inf, np.inf])

In [17]:
N=10
M=4
colind = np.arange(N)
rowind = np.arange(M)

In [18]:
m2 = ConcreteModel()
m2.constraints = ConstraintList()
m2.x = Var(colind)

In [19]:
for i in range(M):
  m2.constraints.add(sum(const_coefs[i][j]*m2.x[j] for j in colind)<=const_rhs[i])

In [20]:
for i in range(N):
  m2.x[i].setlb(lb[i])
  m2.x[i].setub(rb[i])

In [21]:
m2.obj = Objective(expr = sum(obj_coefs[j]*m2.x[j] for j in range(N)), sense = minimize)

In [22]:
m2.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

1 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  -inf :  None :   4.0 : False :  True :  Reals
          1 :  -inf :  None :   inf : False :  True :  Reals
          2 :   0.0 :  None :   inf : False :  True :  Reals
          3 :   0.0 :  None :   2.0 : False :  True :  Reals
          4 : -67.0 :  None :  20.0 : False :  True :  Reals
          5 :   0.0 :  None :  55.0 : False :  True :  Reals
          6 : -23.0 :  None :  23.0 : False :  True :  Reals
          7 :  -5.0 :  None :  15.0 : False :  True :  Reals
          8 :   6.0 :  None :   inf : False :  True :  

In [23]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(m2).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -196.378974895398
  Upper bound: -196.378974895398
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 11
  Number of nonzeros: 29
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.015305519104003906
# ----------------------------------------------------------
#   Solution Information
# ----------

In [24]:
print('Optimal value of objective functn is ', m2.obj())
for i in range(N):
  print("The value of x[",i+1,'] is ',m2.x[i](), sep='')

Optimal value of objective functn is  -196.3789748953975
The value of x[1] is 4.0
The value of x[2] is 5.8347280334728
The value of x[3] is 0.0
The value of x[4] is 0.0
The value of x[5] is -29.3
The value of x[6] is 0.0
The value of x[7] is 23.0
The value of x[8] is 15.0
The value of x[9] is 6.0
The value of x[10] is 1.0


In [25]:
m2.constraints.display()

constraints : Size=4
    Key : Lower : Body               : Upper
      1 :  None : -3.465271966527201 :  35.4
      2 :  None :  -23.6652719665272 :  47.9
      3 :  None :  41.42999999999998 : 41.43
      4 :  None :               22.7 :  22.7


In [26]:
print("Constraint 3 and constraint 4 are active constraints since they satisfy strict equalities")
print("Constraint 1 and constraint 2 are inactive constraints since they don't satisfy strict equalities")

Constraint 3 and constraint 4 are active constraints since they satisfy strict equalities
Constraint 1 and constraint 2 are inactive constraints since they don't satisfy strict equalities
